## Importação das Bibiliotecas

In [31]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine
import os
from pyspark.sql.functions import col, count, sum
import pymssql


## Criando uma SparkSession

In [3]:
spark = SparkSession.builder.remote("sc://spark-connect").getOrCreate()

## Processo de Leitura do CSV de Participantes


In [9]:
#Path dos arquivos

caminho_arquivo_2022 = '/opt/datasets/ENEM/2022/microdados_enem_2022/DADOS/MICRODADOS_ENEM_2022.csv'
caminho_arquivo_2021 = '/opt/datasets/ENEM/2021/microdados_enem_2021/DADOS/MICRODADOS_ENEM_2021.csv'
caminho_arquivo_2020 = '/opt/datasets/ENEM/2020/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv'
caminho_arquivo_2019 = '/opt/datasets/ENEM/2019/microdados_enem_2019/DADOS/MICRODADOS_ENEM_2019.csv'

In [10]:
os.chdir('/opt/datasets/ENEM/2022/microdados_enem_2022/DADOS/')
print(os.getcwd())

/opt/datasets/ENEM/2022/microdados_enem_2022/DADOS


In [12]:
#Função Paa ler os arquivos CSV
def lerCSV(spark, caminho_arquivo):
    df = spark.read.csv(caminho_arquivo, sep=';', header=True, inferSchema=True, encoding='ISO-8859-1')
    #colunas_selecionadas = df.columns[12:19]
    #return df.select(colunas_selecionadas)
    return df


In [13]:
##Chama função quer ler o arquivo CSV e Transforma para DF.SPARK

dfEscola22 = lerCSV(spark, caminho_arquivo_2022)
#dfEscola21 = lerCSV(spark, caminho_arquivo_2021)
#dfEscola20 = lerCSV(spark, caminho_arquivo_2020)
#dfEscola19 = lerCSV(spark, caminho_arquivo_2019)


In [15]:
#Unir os DF em um só 

#dfDadosGerais = dfEscola22.union(dfEscola21).union(dfEscola20).union(dfEscola19)
dfDadosGerais = dfEscola22


In [17]:
dfPreencher = dfDadosGerais.fillna(-2)  # Preenche com -2(sem ref), por exemplo
dfPreencher.cache()

DataFrame[NU_INSCRICAO: bigint, NU_ANO: int, TP_FAIXA_ETARIA: int, TP_SEXO: string, TP_ESTADO_CIVIL: int, TP_COR_RACA: int, TP_NACIONALIDADE: int, TP_ST_CONCLUSAO: int, TP_ANO_CONCLUIU: int, TP_ESCOLA: int, TP_ENSINO: int, IN_TREINEIRO: int, CO_MUNICIPIO_ESC: int, NO_MUNICIPIO_ESC: string, CO_UF_ESC: int, SG_UF_ESC: string, TP_DEPENDENCIA_ADM_ESC: int, TP_LOCALIZACAO_ESC: int, TP_SIT_FUNC_ESC: int, CO_MUNICIPIO_PROVA: int, NO_MUNICIPIO_PROVA: string, CO_UF_PROVA: int, SG_UF_PROVA: string, TP_PRESENCA_CN: int, TP_PRESENCA_CH: int, TP_PRESENCA_LC: int, TP_PRESENCA_MT: int, CO_PROVA_CN: int, CO_PROVA_CH: int, CO_PROVA_LC: int, CO_PROVA_MT: int, NU_NOTA_CN: double, NU_NOTA_CH: double, NU_NOTA_LC: double, NU_NOTA_MT: double, TX_RESPOSTAS_CN: string, TX_RESPOSTAS_CH: string, TX_RESPOSTAS_LC: string, TX_RESPOSTAS_MT: string, TP_LINGUA: int, TX_GABARITO_CN: string, TX_GABARITO_CH: string, TX_GABARITO_LC: string, TX_GABARITO_MT: string, TP_STATUS_REDACAO: int, NU_NOTA_COMP1: int, NU_NOTA_COMP2:

In [25]:
dfPreencherDistinct = dfPreencher.select(
    col("NU_INSCRICAO").alias("NU_INSCRICAO"),
    col("NU_ANO").alias("ANO_EDICAO"),
    col("TP_ANO_CONCLUIU").alias("ANO_CONCLUSAO"),
    col("TP_SEXO").alias("SEXO_CODIGO"),
    col("CO_MUNICIPIO_ESC").alias("MUNICIPIO_CODIGO"),
    col("TP_ESTADO_CIVIL").alias("ESTADO_CIVIL_ID"),
    col("TP_COR_RACA").alias("COR_RACA_ID"),
    col("TP_NACIONALIDADE").alias("NACIONALIDADE_ID"),
    col("TP_ST_CONCLUSAO").alias("SITUACAO_ESCOLARIDADE_ID"),
    col("TP_ESCOLA").alias("ESCOLA_ID"),
    col("Q006").alias("FAIXA_RENDA_MENSAL_ID"),
    col("TP_FAIXA_ETARIA").alias("FAIXA_ETARIA_ID"),
    col("NU_NOTA_CN").alias("NOTA_CIENCIA_DA_NATUREZA"),
    col("NU_NOTA_CH").alias("NOTA_CIENCIA_DA_HUMANA"),
    col("NU_NOTA_LC").alias("NOTA_LINGUAGEM_CODIGO"),
    col("NU_NOTA_MT").alias("NOTA_MATEMATICA")
).distinct()
dfPreencherDistinct.cache()

DataFrame[NU_INSCRICAO: bigint, ANO_EDICAO: int, ANO_CONCLUSAO: int, SEXO_CODIGO: string, MUNICIPIO_CODIGO: int, ESTADO_CIVIL_ID: int, COR_RACA_ID: int, NACIONALIDADE_ID: int, SITUACAO_ESCOLARIDADE_ID: int, ESCOLA_ID: int, FAIXA_RENDA_MENSAL_ID: string, FAIXA_ETARIA_ID: int, NOTA_CIENCIA_DA_NATUREZA: double, NOTA_CIENCIA_DA_HUMANA: double, NOTA_LINGUAGEM_CODIGO: double, NOTA_MATEMATICA: double]

In [43]:
dfAgregate = dfPreencherDistinct.select(
    "ANO_EDICAO",
    "ANO_CONCLUSAO",
    "SEXO_CODIGO",
    "MUNICIPIO_CODIGO",
    "ESTADO_CIVIL_ID",
    "COR_RACA_ID",
    "NACIONALIDADE_ID",
    "SITUACAO_ESCOLARIDADE_ID",
    "ESCOLA_ID",
    "FAIXA_RENDA_MENSAL_ID",
    "FAIXA_ETARIA_ID",
    "NU_INSCRICAO",  # Adicionando essa coluna para o count
    "NOTA_CIENCIA_DA_NATUREZA",
    "NOTA_CIENCIA_DA_HUMANA",
    "NOTA_LINGUAGEM_CODIGO",
    "NOTA_MATEMATICA"
)

# Agregação após o agrupamento
dfAgregate = dfAgregate.groupBy(
    "ANO_EDICAO",
    "ANO_CONCLUSAO",
    "SEXO_CODIGO",
    "MUNICIPIO_CODIGO",
    "ESTADO_CIVIL_ID",
    "COR_RACA_ID",
    "NACIONALIDADE_ID",
    "SITUACAO_ESCOLARIDADE_ID",
    "ESCOLA_ID",
    "FAIXA_RENDA_MENSAL_ID",
    "FAIXA_ETARIA_ID"
).agg(
    count("NU_INSCRICAO").alias("QTD"),
    sum("NOTA_CIENCIA_DA_NATUREZA").alias("NOTA_CIENCIA_DA_NATUREZA"),
    sum("NOTA_CIENCIA_DA_HUMANA").alias("NOTA_CIENCIA_DA_HUMANA"),
    sum("NOTA_LINGUAGEM_CODIGO").alias("NOTA_LINGUAGEM_CODIGO"),
    sum("NOTA_MATEMATICA").alias("NOTA_MATEMATICA")
)

In [ ]:
dfAgregate.count()

# Salvar em SQL 
Salvar os dados em uma tabela SQL


## Parametrizando a conexão do Banco SQL SERVER Usando JDBC

In [35]:
server_name = "46aa1e5a9d50"
database_name = "DockerEnem"
username = "sa"
password = "bi@123456"

In [ ]:
# Nome da tabela a ser excluída
table_name = "ODS_ENEM"

#obter Schema do dfAgregate 
df_schema = dfAgregate.schema

# Gerando as definições de coluna para a tabela
columns_definitions = [f"{field.name} {field.dataType}" for field in df_schema.fields]

# Mapeamento de tipos de dados do Spark para tipos de dados do SQL Server
data_type_mapping = {
    "IntegerType": "INT",
    "StringType": "NVARCHAR(255)",  # Você pode ajustar o tamanho conforme necessário
    "DoubleType": "FLOAT",
    "LongType": "BIGINT"
}

# Gerando as definições de coluna para a tabela com os tipos de dados corretos
columns_definitions = [f"{field.name} {data_type_mapping.get(str(field.dataType), 'NVARCHAR(255)')}" for field in df_schema.fields]


# Combinando as definições de coluna em uma string
columns_str = ",\n".join(columns_definitions)

In [50]:
# Conectando ao banco de dados
conn = pymssql.connect(server=server_name, database=database_name, user=username, password=password)

# Criando um cursor
cursor = conn.cursor()

In [53]:
# Dropando a tabela se ela já existir
drop_table_query = """
IF OBJECT_ID('ODS_ENEM', 'U') IS NOT NULL
    DROP TABLE ODS_ENEM
"""
cursor.execute(drop_table_query)


SparkConnectGrpcException: (org.apache.spark.SparkException) Job aborted due to stage failure: ResultStage 75 (executePlan at SparkConnectServiceGrpc.java:449) has failed the maximum allowable number of times: 4. Most recent failure reason:
org.apache.spark.shuffle.MetadataFetchFailedException: Missing an output location for shuffle 3 partition 171
	at org.apache.spark.MapOutputTracker$.validateStatus(MapOutputTracker.scala:1718)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11(MapOutputTracker.scala:1665)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11$adapted(MapOutputTracker.scala:1664)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at org.apache.spark.MapOutputTracker$.convertMapStatuses(MapOutputTracker.scala:1664)
	at org.apache.spark.MapOutputTrackerWorker.getMapSizesByExecutorIdImpl(MapOutputTracker.

In [ ]:
# Criando o comando CREATE TABLE
create_table_query = f"""
CREATE TABLE {table_name} (
{columns_str}
)
"""
# Executando o comando SQL para criar a tabela
cursor.execute(create_table_query)

In [ ]:
# Inserindo os dados na tabela
insert_query = f"""
INSERT INTO ODS_ENEM VALUES {",".join(["(" + ",".join(map(str, row)) + ")" for row in dfAgregate.collect()])}
"""
cursor.execute(insert_query)

In [ ]:
# Commit das alterações no banco de dados
conn.commit()

In [ ]:
# Fechando a conexão
conn.close()

In [ ]:
print(f'Tabela {nomeTabela} criada com sucesso no banco de dados.')
